<a href="https://colab.research.google.com/github/g8428/TIL/blob/master/220308_nlp_DACON.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd

# 데이터 로드 - 판다스

In [ ]:
train_path = '/content/drive/MyDrive/2022-1/국비지원교육/실습폴더/ratings_train.txt'
with open(train_path, 'r') as f:
    train = f.readlines()

train_li = []
for a in train:
    a = a.split('\t')
    a[-1] = a[-1][0]
    train_li.append(a)

train = pd.DataFrame(data=train_li[1:], columns = train_li[0])
x_train = train[['id','document']]
y_train = train[['l']]

test_path = '/content/drive/MyDrive/2022-1/국비지원교육/실습폴더/ratings_test.txt'
with open(test_path, 'r') as f:
    test = f.readlines()

test_li = []
for a in test:
    a = a.split('\t')
    a[-1] = a[-1][0]
    test_li.append(a)

test = pd.DataFrame(data=test_li[1:], columns = test_li[0])
x_test = test[['id','document']]
y_test = test[['l']]

y_train['l'] = list(map(int,y_train['l']))
y_test['l'] = list(map(int,y_test['l']))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


# Kiwipiepy 형태소 분석기

In [ ]:
# !pip install --upgrade pip
# !pip install kiwipiepy

In [ ]:
# !python3 -m kiwipiepy

In [ ]:
from kiwipiepy import Kiwi, Option
from kiwipiepy.utils import Stopwords

# 키위파이 설명

In [ ]:
# 키위파이 링크 https://github.com/bab2min/kiwipiepy


# Kiwi(num_workers=0, model_path=None, options=Option.LOAD_DEFAULT_DICTIONARY | Option.INTEGRATE_ALLOMORPH, load_default_dict=True, integrate_allomorph=True)


kiwi = Kiwi(options = Option.INTEGRATE_ALLOMORPH) # 아/어/었 등 음운 통합
stopwords = Stopwords()
# stopwords.add(('10','SN'))

In [ ]:
tokens=kiwi.tokenize(x_train.document[10], stopwords = stopwords,
                     normalize_coda=True) # 이상한 받침 제거

tokens_list=[t.form for t in tokens if t.tag in {'MM','NNG','NP','NNP','VV','VA','VX','IC','MAG','MAJ','MM'}]

In [ ]:
tokens_list

['걍인피니트가짱', '진짜', '짱']

In [ ]:
sent_tokens = kiwi.split_into_sents(x_train.document[10],return_tokens=False) # 문장분리
tokens_list=[]
for t in sent_tokens:
    tokens_list.append(t.text)

tokens_list

['걍인피니트가짱이다.', '진짜짱이다♥']

# 트레인 데이터 토큰화

In [ ]:
kiwi = Kiwi()
stopwords = Stopwords()
train_tokens = []
for sentence in x_train.document:
    tokens = kiwi.tokenize(sentence,stopwords=stopwords,normalize_coda=True)
    temp = [t.form for t in tokens if t.tag[0] in {'M','N','V','I'}]
    train_tokens.append(temp)
x_train['tokens'] = np.array(train_tokens)
train_labels = np.array(y_train)

# 테스트데이터 토큰화

In [ ]:
kiwi = Kiwi()
stopwords = Stopwords()
test_tokens = []
for sentence in x_test.document:
    tokens = kiwi.tokenize(sentence,stopwords=stopwords,normalize_coda=True)
    temp = [t.form for t in tokens if t.tag[0] in {'M','N','V','I'}]
    test_tokens.append(temp)

x_test['tokens'] = np.array(test_tokens)
test_labels = np.array(y_test)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  if __name__ == '__main__':


# 텐서플로, 시퀀스화, 패딩

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import tensorflow as tf

# 모델 생성

In [ ]:
# 훈련시작
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Embedding, GlobalAveragePooling1D, Dense

vocab_size = 25000
embedding_dim = 32 # 각 단어 하나당 임베딩 차원
max_length = 100 # 단어 개수 = 인풋 디멘션
trunc_type = 'post' # 뒤에서 자르기
oov_tok = '<oov>' # oov 처리

model = Sequential()
# 원핫인코딩 사이즈, 원핫벡터를 덴스벡터로 만들때의 차원수
# 단어 수가 10000개 원핫 -> 16차원에 임베딩 시키는 것
model.add(Embedding(vocab_size, embedding_dim ))

# 히든계층 - 풀링, 활성함수
model.add(GlobalAveragePooling1D())
model.add(Dense(6, activation='relu'))

# 아웃풋 계층 - 시그모이드, 크로스엔트로피
model.add(Dense(1, activation='sigmoid')) # 감성분석 이진분류를 위해 시그모이드

model.compile(loss = 'binary_crossentropy', optimizer = 'adam',
              metrics=['accuracy'])

model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, None, 32)          800000    
                                                                 
 global_average_pooling1d_4   (None, 32)               0         
 (GlobalAveragePooling1D)                                        
                                                                 
 dense_8 (Dense)             (None, 6)                 198       
                                                                 
 dense_9 (Dense)             (None, 1)                 7         
                                                                 
Total params: 800,205
Trainable params: 800,205
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# 토크나이저
tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(x_train.tokens)
# vocab 확인, 시퀀스화, 패딩
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(x_train.tokens)
padded = pad_sequences(sequences, maxlen=max_length, truncating=trunc_type)

In [ ]:
# 테스트 데이터도 시퀀스로
test_sequences = tokenizer.texts_to_sequences(x_test.tokens)
test_padd = pad_sequences(test_sequences, maxlen=max_length, truncating=trunc_type)

# 모델 학습

In [ ]:
num_epochs = 100
batch_size = 1500
hist = model.fit(padded, train_labels, validation_data=(test_padd,test_labels),
                 epochs = num_epochs, batch_size = batch_size)

Epoch 1/100
100/100 [==============================] - 2s 22ms/step - loss: 0.6698 - accuracy: 0.6774 - val_loss: 0.6498 - val_accuracy: 0.7200
Epoch 2/100
100/100 [==============================] - 1s 13ms/step - loss: 0.6202 - accuracy: 0.7505 - val_loss: 0.5952 - val_accuracy: 0.7541
Epoch 3/100
100/100 [==============================] - 1s 13ms/step - loss: 0.5649 - accuracy: 0.7853 - val_loss: 0.5473 - val_accuracy: 0.7866
Epoch 4/100
100/100 [==============================] - 1s 13ms/step - loss: 0.5203 - accuracy: 0.8064 - val_loss: 0.5134 - val_accuracy: 0.7936
Epoch 5/100
100/100 [==============================] - 1s 14ms/step - loss: 0.4871 - accuracy: 0.8175 - val_loss: 0.4878 - val_accuracy: 0.8060
Epoch 6/100
100/100 [==============================] - 1s 13ms/step - loss: 0.4619 - accuracy: 0.8244 - val_loss: 0.4691 - val_accuracy: 0.8147
Epoch 7/100
100/100 [==============================] - 1s 13ms/step - loss: 0.4427 - accuracy: 0.8302 - val_loss: 0.4566 - val_accuracy:

In [ ]:
model.evaluate(test_padd, test_labels)

1563/1563 [==============================] - 4s 3ms/step - loss: 0.5210 - accuracy: 0.8003


[0.5209634900093079, 0.800320029258728]

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(train[['document']],train[['l']], test_size=0.2, random_state=42)